In [ ]:
from pathlib import Path
from collections import defaultdict

from IPython.display import Image

# Use this to read in a CCD image
from astropy.nddata import CCDData

# Use this to respresent the coordinates of the variable star
from astropy.coordinates import SkyCoord

# We will use table to read and write tables of data as CSV files (or as FITS)
from astropy.table import Table

# We will use this to get the APASS stars in the field of view of an image
from stellarphot import apass_dr9

import requests
import pandas as pd
import json

In [ ]:
# Star chart parameters
var_star_name = "V0533 Her"

# File for saving comparison star magnitudes in
apass_mags_comp_stars = "apass-V0533-Her.csv"

# You chould not need to change these....
fov = 40
limiting_magnitude = 16

In [ ]:
payload = {
    "format": "json",
    "star": var_star_name,
    "fov": fov,
    "maglimit": limiting_magnitude,
}

r = requests.get("https://app.aavso.org/vsp/api/chart", params=payload)

print(r.url)

aavso_response = r.json()

## L:ook at the chart below and decide which star will be your "check star"

### Write down its label (the 2 or 3 digit number near the star)

In [ ]:
# Chop off the format argument at the end
image_uri = aavso_response["image_uri"].split("?")[0]
Image(image_uri)

In [ ]:
var_coord = SkyCoord(
    ra=aavso_response["ra"], dec=aavso_response["dec"], unit=("hour", "degree")
)

In [ ]:
if not aavso_response["photometry"]:
    raise RuntimeError(
        "No comparison stars for this variable -- request them from the AAVSO"
    )

In [ ]:
json = json.dumps(aavso_response["photometry"])
panda = pd.read_json(json, orient="records")

In [ ]:
panda

In [ ]:
aavso_comps = Table.from_pandas(panda)

In [ ]:
aavso_comps["coords"] = SkyCoord(
    ra=aavso_comps["ra"], dec=aavso_comps["dec"], unit=("hour", "degree")
)

In [ ]:
apass = apass_dr9(var_coord)

apass["coords"] = SkyCoord(
    ra=apass["RAJ2000"], dec=apass["DEJ2000"], unit=("degree", "degree")
)

## Match the comp stars to APASS stars

One of the columns of the APASS table will be called `coords` and will be a list of coordinate objects. Astropy coordinates have the magical ability to match themselves to the coordinates of another set of coordinats.

You could use either the full apass list or the low error one below.

In [ ]:
index, d2d, _ = aavso_comps["coords"].match_to_catalog_sky(apass["coords"])

In [ ]:
if (d2d.arcsec > 1.5).any():
    raise RuntimeError("Coordinate mismatch")

In [ ]:
apass_comp_stars = apass[index]

In [ ]:
apass_comp_stars["auid"] = aavso_comps["auid"]
apass_comp_stars["label"] = aavso_comps["label"]

In [ ]:
apass_mag_cols_band_map = dict(Vmag="V", Bmag="B", g_mag="SG", r_mag="SR", i_mag="SI")

mag_err_cols = {k: f"e_{k}" for k in apass_mag_cols_band_map.keys()}

In [ ]:
del apass_comp_stars["coords"], apass_comp_stars["B-V"], apass_comp_stars["e_B-V"]

In [ ]:
constant_cols = (
    set(apass_comp_stars.colnames)
    - set(apass_mag_cols_band_map.keys())
    - set(mag_err_cols.values())
)

In [ ]:
rearranged = defaultdict(list)
n_bands = len(apass_mag_cols_band_map.keys())

for row in apass_comp_stars:
    for cc in constant_cols:
        rearranged[cc].extend([row[cc]] * n_bands)
    for band, band_name in apass_mag_cols_band_map.items():
        rearranged["band"].append(band_name)
        rearranged["mag"].append(row[band])
        rearranged["mag_err"].append(row[mag_err_cols[band]])

In [ ]:
rearranged = Table(rearranged)

In [ ]:
rearranged

In [ ]:
rearranged.write(apass_mags_comp_stars, overwrite=True)

## Write AAVSO comparison magnitudes (from VSD)

Use similar logic to `rearranged` code above to get a flat table.